In [1]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

In [2]:
train_data = pd.read_csv('./data/titanic/train.csv')
test_data = pd.read_csv('./data/titanic/test.csv')

In [3]:
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,493,0,1,"Molson, Mr. Harry Markland",male,55.0,0,0,113787,30.5000,C30,S
1,53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C
2,388,1,2,"Buss, Miss. Kate",female,36.0,0,0,27849,13.0000,NaN,S
3,192,0,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S
4,687,0,3,"Panula, Mr. Jaako Arnold",male,14.0,4,1,3101295,39.6875,NaN,S


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          566 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        168 non-null    object 
 11  Embarked     710 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.9+ KB


In [25]:
df_Sex = pd.get_dummies(train_data['Sex'])
df_Sex.head()

,female,male
0,0,1
1,1,0
2,1,0
3,0,1
4,0,1


In [32]:
def proprecesing(data):
    df_result = pd.DataFrame()
    
    df_Pclass = pd.get_dummies(data['Pclass'])
    df_Pclass.columns = ['Pclass_' + str(x) for x in df_Pclass.columns]
    df_result = pd.concat([df_result, df_Pclass], axis = 1)
    
    df_Embarked = pd.get_dummies(data['Embarked'])
    df_Embarked.columns = ['Embarked_' + str(x) for x in df_Embarked.columns]
    df_result = pd.concat([df_result, df_Embarked], axis = 1)
    
    df_Sex = pd.get_dummies(data['Sex'])
#     df_Sex.columns = ['Sex_' + str(x) for x in df_Sex.columns]
    df_result = pd.concat([df_result, df_Sex], axis=1)
    
    df_result['SibSp'] = data['SibSp']
    df_result['Parch'] = data['Parch']
    df_result['Fare'] = data['Fare']
    
    df_result['Age'] = data['Age'].fillna(data['Age'].mean())
    
    df_result['Cabin_null'] =  pd.isna(data['Cabin']).astype('int32')
    
    
    return df_result
    

In [33]:
train_x = proprecesing(train_data)
train_y = train_data['Survived']

In [34]:
train_x.head()

,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,female,male,SibSp,Parch,Fare,Age,Cabin_null
0,1,0,0,0,0,1,0,1,0,0,30.5000,55.0,0
1,1,0,0,1,0,0,1,0,1,0,76.7292,49.0,0
2,0,1,0,0,0,1,1,0,0,0,13.0000,36.0,1
3,0,1,0,0,0,1,0,1,0,0,13.0000,19.0,1
4,0,0,1,0,0,1,0,1,4,1,39.6875,14.0,1


In [35]:
train_y.head()

0    0
1    1
2    1
3    0
4    0
Name: Survived, dtype: int64

In [36]:
test_x = proprecesing(test_data)
test_y = test_data['Survived']

In [38]:
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(train_x, train_y)
print(tpot.score(test_x, test_y))
tpot.export('tpot_iris_pipeline.py')

C:\Users\wuyao\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\wuyao\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\wuyao\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



Generation 1 - Current best internal CV score: 0.8187826258248794
Generation 2 - Current best internal CV score: 0.8187826258248794
Generation 3 - Current best internal CV score: 0.828631931448833
Generation 4 - Current best internal CV score: 0.8314586821629074
Generation 5 - Current best internal CV score: 0.8314586821629074
Best pipeline: MLPClassifier(GradientBoostingClassifier(BernoulliNB(input_matrix, alpha=10.0, fit_prior=False), learning_rate=0.01, max_depth=10, max_features=0.9000000000000001, min_samples_leaf=12, min_samples_split=4, n_estimators=100, subsample=0.4), alpha=0.0001, learning_rate_init=0.001)
0.8100558659217877


In [50]:
import sklearn.svm as svm
import sklearn

In [61]:
clf = svm.SVC(kernel='linear')

In [62]:
clf.fit(train_x, train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [63]:
score = sklearn.metrics.accuracy_score(test_y, clf.predict(test_x), normalize=True, sample_weight=None)
score

0.770949720670391